In [ ]:
pip install --upgrade pip

In [ ]:
!pip install pinecone-client

In [3]:
from pinecone import Pinecone
import os

In [4]:
# initialize connection to pinecone (get API key at app.pc.io)
api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'
environment = os.environ.get('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'

# configure client
pc = Pinecone(api_key=api_key)

In [5]:
index = pc.Index(host="https://company-profiles-9atubxs.svc.gcp-starter.pinecone.io")

index_stats_response = index.describe_index_stats()
print(index_stats_response)

{'dimension': 1,
 'index_fullness': 1e-05,
 'namespaces': {'': {'vector_count': 1}},
 'total_vector_count': 1}


In [21]:
# List indexes
pc.list_indexes()

{'indexes': [{'dimension': 1,
              'host': 'company-profiles-9atubxs.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'company-profiles',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [24]:
# Describe index
pc.describe_index("company-profiles")

{'dimension': 1,
 'host': 'company-profiles-9atubxs.svc.gcp-starter.pinecone.io',
 'metric': 'cosine',
 'name': 'company-profiles',
 'spec': {'pod': {'environment': 'gcp-starter',
                  'pod_type': 'starter',
                  'pods': 1,
                  'replicas': 1,
                  'shards': 1}},
 'status': {'ready': True, 'state': 'Ready'}}

In [67]:
#create index
new_index_name = "stock-prices"
dimension = 3
metric = "cosine"

In [68]:
# if exists delete
for index in pc.list_indexes().indexes:
    if index.name == new_index_name:
        pc.delete_index(new_index_name)

In [69]:
from pinecone import PodSpec
pc.create_index(
    name=new_index_name, 
    dimension=dimension, 
    metric=metric, 
    spec=PodSpec(
        environment="gcp-starter",
        pod_type="starter",
        pods=1,
        replicas=1,
        shards=1
))

In [70]:
pc.describe_index(new_index_name)

{'dimension': 3,
 'host': 'stock-prices-9atubxs.svc.gcp-starter.pinecone.io',
 'metric': 'cosine',
 'name': 'stock-prices',
 'spec': {'pod': {'environment': 'gcp-starter',
                  'pod_type': 'starter',
                  'pods': 1,
                  'replicas': 1,
                  'shards': 1}},
 'status': {'ready': True, 'state': 'Ready'}}

In [78]:
index = pc.Index(name=new_index_name)

In [79]:
index